# Online Production

In [26]:
#!pip install -U hopsworks --quiet

In [27]:
import warnings
warnings.filterwarnings("ignore")

In [28]:
import hopsworks

project = hopsworks.login()

fs = project.get_feature_store()
ms = project.get_model_serving()


Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/550038
Connected. Call `.close()` to terminate connection gracefully.
Connected. Call `.close()` to terminate connection gracefully.


In [29]:
# Fetch Model from Hopsworks Model Registry
deployment = ms.get_deployment(name="beerratingmodeldeployment1")
deployment.start()

Deployment is already running


## 🚀 Send an Inference Request

In [30]:
# Send inference requests to the deployed model as follows:
feature_view = fs.get_feature_view("user_beer_feature_view", 1)

In [31]:
# Initialize serving
feature_view.init_serving(1)    

2024-05-07 07:04:56,865 INFO: Default Online Store Client is set to sql.
2024-05-07 07:04:57,918 INFO: Initialising Vector Server Online SQL client
2024-05-07 07:05:01,257 INFO: Default Online Store Client is set to sql.
2024-05-07 07:05:02,235 INFO: Initialising Vector Server Online SQL client
